In [1]:
from pyscipopt import Model,quicksum, multidict
import numpy as np
import pandas as pd
import openpyxl
import datetime as dt

In [2]:
data = pd.read_excel('C:\\Users\\liuju\\Desktop\\readcode\\ljtext\\LiNingAllData.xlsx')

In [3]:
# data

In [4]:
shoes_data = data[data['class_0'].isin(['鞋'])]

In [5]:
shoes_data = shoes_data.drop(['org_name','class_0'],axis=1).reset_index().drop('index',axis=1)

In [6]:
shoes_data

,class_2,size,org_id,qty,ratio
0,减震跑鞋,10,A11001A,45,0.035239
1,减震跑鞋,10.5,A11001A,447,0.350039
2,减震跑鞋,11,A11001A,144,0.112764
3,减震跑鞋,11.5,A11001A,0,0.000000
4,减震跑鞋,12,A11001A,89,0.069695
5,减震跑鞋,12.5,A11001A,0,0.000000
6,减震跑鞋,13,A11001A,0,0.000000
7,减震跑鞋,13.5,A11001A,0,0.000000
8,减震跑鞋,14,A11001A,0,0.000000
9,减震跑鞋,4.5,A11001A,0,0.000000


In [7]:
class_list = shoes_data['class_2'].drop_duplicates().reset_index().drop('index',axis =1)

In [8]:
class_arry = class_list['class_2'].values

In [9]:
class_arry

array(['减震跑鞋', '户外休闲鞋', '潮流休闲鞋', '潮流拖鞋', '灵动跑鞋', '灵活训练鞋', '稳定跑鞋',
       '篮球专业比赛鞋', '篮球休闲鞋', '篮球场地鞋', '经典休闲鞋', '缓震训练鞋', '羽毛球专业鞋', '羽毛球训练鞋',
       '街头潮流休闲鞋', '轻质跑鞋'], dtype=object)

In [10]:
def cal_shoes_model(data_result,minput):
    
    data_result = data_result.reset_index().set_index(['class_2','org_id','size'])
    
#     minput = 18
#     minput = 12
    
    #建立模型
    model = Model("LiNingExample")

    #  变量及参数定义
    M = 100000   #极大值
    diffabs = {} #绝对值
    bl = {} #bool
    X = {} #决策变量
    Xmax = {} # X 的最大值
    dllb = {}
    diff = {}
    
    dfx = data_result.reset_index('org_id')
    # 决策变量X，每个尺码的件数
    for idx_s in dfx.index.unique():
        X[idx_s] = model.addVar(vtype="I", name="X")
    #辅助变量 max 尺码件数X中的最大值Xmax
    dfmax = data_result.reset_index(['size','org_id'])
    
    for idx_s in dfx.index.unique():
        class_2, size = idx_s
        if class_2 not in Xmax:
            Xmax[class_2] = model.addVar(vtype ='I', name = "Xmax")
        bl[idx_s] = model.addVar(vtype="B", name="bl")

        model.addCons(Xmax[class_2] - X[idx_s] >= 0)
        model.addCons(X[idx_s] - Xmax[class_2] + M * (1 - bl[idx_s])>= 0 )
        model.addCons(X[idx_s] - Xmax[class_2] - M * bl[idx_s] <= -1 )
    
    #辅助等式  绝对值计算 及约束确定
    
    for idx in data_result.index.unique():
        myratio = data_result.loc[idx,'ratio']
        # print('myration:', myratio)
        dllb[idx] = model.addVar(vtype = 'B', name = 'dllb')
        diff[idx] = model.addVar(lb = None,  vtype = 'C', name = 'diff')
        diffabs[idx] = model.addVar(vtype = 'C', name = 'diffabs')
#         diffabs[idx] = model.addVar(vtype = 'C', name = 'diffabs', obj = data_result.loc[idx, 'qty'])
    
    #设置目标
    for idx in data_result.index.unique():
        model.setObjective(data_result.loc[idx, 'qty']*diffabs[idx], clear = False)

    for idx in data_result.index.unique():
        myratio = data_result.loc[idx,'ratio']

        class_2, org_id, size = idx
        model.addCons(diff[idx] == X[class_2, size] - myratio * Xmax[class_2])

        model.addCons(diffabs[idx] >= diff[idx])
        model.addCons(diffabs[idx] >= -1 * diff[idx])
        model.addCons(diffabs[idx] <= diff[idx] + M * (1 - dllb[idx]))
        model.addCons(diffabs[idx] <= -1 * diff[idx] + M * dllb[idx])
    
    df = data_result.reset_index('size')
    df_1 = data_result.reset_index(['org_id','size'])
    for idx in df_1.index.unique():
        class_2 = idx
        model.addCons(quicksum(bl[class_2, size_1] for size_1 in df_1.loc[idx, ['size']].drop_duplicates().values.ravel()) >= 1)
    
    # 约束： 每个尺码的件数之和 等于 最小装箱数
    for idx_s in df.index.unique():
        class_2, org_id = idx_s
        model.addCons(quicksum(X[class_2,size_1] for size_1 in df.loc[idx_s, ['size']].values.ravel()) == minput)

    
    model.setMinimize()
    model.optimize()
    status = model.getStatus()
    
    # 输出到出X为excle到本地
    OutX = {}
    if model.getStatus() =='optimal':
        for i in X:
            OutX[i] = round(model.getVal(X[i]))
    else:
        print("status",'*'*30, model.getStatus())
        print("obj:", model.getObjVal())
        print("gap:", model.getGap())
#         break
    return OutX
        
#     print(OutX)
#     print(liningResult)
    
#     return OutX

In [11]:
def load_shoes_result(result, minput):
    exl_time = dt.datetime.now().strftime('%Y%m%d%H%M')
    locaurl ='C:\\Users\\liuju\\Desktop\\LiNingResult\\LiningShoesResult-'+str(minput)+'双'+exl_time+'.xlsx'
    result.to_excel(locaurl, index = False)
    print('有解，且X已输出到本地') 

In [12]:
shoes_result6 =pd.DataFrame(columns=('小类','尺码','6尺码件数'))
shoes_result12 =pd.DataFrame(columns=('小类','尺码','12尺码件数'))

minput12 = 12
minput6 = 6

for class_name in class_arry:
    shoes_data_test12 = shoes_data.loc[shoes_data['class_2']== class_name]
    shoes_data_test12 = shoes_data_test12.reset_index().drop('index',axis=1)
   
    class_result = {}
    class_result = cal_shoes_model(shoes_data_test12,minput12)
    liningResult = pd.DataFrame(class_result, index =['12尺码件数']).T.rename_axis(['小类','尺码']).reset_index()
    shoes_result12 = pd.concat([shoes_result12,liningResult],ignore_index=True)
load_shoes_result(shoes_result12,minput12)

有解，且X已输出到本地


In [13]:
for class_name in class_arry:
    shoes_data_test = shoes_data.loc[shoes_data['class_2']== class_name]
    shoes_data_test = shoes_data_test.reset_index().drop('index',axis=1)
    class_result = {}
    class_result = cal_shoes_model(shoes_data_test,minput6)
    liningResult = pd.DataFrame(class_result, index =['6尺码件数']).T.rename_axis(['小类','尺码']).reset_index()
    shoes_result6 = pd.concat([shoes_result6,liningResult],ignore_index=True)
load_shoes_result(shoes_result6, minput6)

有解，且X已输出到本地


In [14]:
shoes_result6

,小类,尺码,6尺码件数
0,减震跑鞋,10,0
1,减震跑鞋,10.5,0
2,减震跑鞋,11,0
3,减震跑鞋,11.5,1
4,减震跑鞋,12,0
5,减震跑鞋,12.5,0
6,减震跑鞋,13,0
7,减震跑鞋,13.5,1
8,减震跑鞋,14,0
9,减震跑鞋,4.5,0


In [15]:
shoes_result12

,小类,尺码,12尺码件数
0,减震跑鞋,10,0
1,减震跑鞋,10.5,1
2,减震跑鞋,11,0
3,减震跑鞋,11.5,0
4,减震跑鞋,12,0
5,减震跑鞋,12.5,0
6,减震跑鞋,13,0
7,减震跑鞋,13.5,1
8,减震跑鞋,14,0
9,减震跑鞋,4.5,0


In [16]:
result = pd.merge(shoes_result6, shoes_result12, how='right', on=['小类', '尺码'])

In [17]:
result

,小类,尺码,6尺码件数,12尺码件数
0,减震跑鞋,10,0,0
1,减震跑鞋,10.5,0,1
2,减震跑鞋,11,0,0
3,减震跑鞋,11.5,1,0
4,减震跑鞋,12,0,0
5,减震跑鞋,12.5,0,0
6,减震跑鞋,13,0,0
7,减震跑鞋,13.5,1,1
8,减震跑鞋,14,0,0
9,减震跑鞋,4.5,0,0


In [18]:
# exl_time = dt.datetime.now().strftime('%Y%m%d%H%M')
# locaurl ='C:\\Users\\liuju\\Desktop\\LiNingResult\\LiningShoesResult-6&12双'+exl_time+'.xlsx'
# result.to_excel(locaurl, index = False)

In [19]:
def cal_shoes_model_2(data_result):
    
    data_result = data_result.reset_index().set_index(['class_2','org_id','size'])
    
#     minput = 18
#     minput = 12
    
    #建立模型
    model = Model("LiNingExample")

    #  变量及参数定义
    M = 100000   #极大值
    diffabs = {} #绝对值
    bl = {} #bool
    X = {} #决策变量
    Xmax = {} # X 的最大值
    dllb = {}
    diff = {}
    
    dfx = data_result.reset_index('org_id')
    
    #决策变量X，每个尺码的比例
    for idx_s in dfx.index.unique():
        X[idx_s] = model.addVar(vtype="C", name="X")
    #辅助变量 max 尺码件数X中的最大值Xmax
    
#     dfmax = data_result.reset_index(['size','org_id'])
    
#     for idx_s in dfx.index.unique():
#         class_2, size = idx_s
#         if class_2 not in Xmax:
#             Xmax[class_2] = model.addVar(vtype ='I', name = "Xmax")
#         bl[idx_s] = model.addVar(vtype="B", name="bl")

#         model.addCons(Xmax[class_2] - X[idx_s] >= 0)
#         model.addCons(X[idx_s] - Xmax[class_2] + M * (1 - bl[idx_s])>= 0 )
#         model.addCons(X[idx_s] - Xmax[class_2] - M * bl[idx_s] <= -1 )
    
    #辅助等式  绝对值计算 及约束确定
    for idx in data_result.index.unique():
        myratio = data_result.loc[idx,'ratio']
        # print('myration:', myratio)
#         if myratio!=0:
#             print(idx,'==========',myratio)
        dllb[idx] = model.addVar(vtype = 'B', name = 'dllb')
        diff[idx] = model.addVar(lb = None,  vtype = 'C', name = 'diff')
        diffabs[idx] = model.addVar(vtype = 'C', name = 'diffabs')
#       diffabs[idx] = model.addVar(vtype = 'C', name = 'diffabs', obj = data_result.loc[idx, 'qty'])
    
    #设置目标

    for idx in data_result.index.unique():
        model.setObjective(data_result.loc[idx, 'qty']*diffabs[idx], clear = False)
#     for idx in data_result.index.unique():
#         model.setObjective(data_result.loc[idx, 'qty']*diffabs[idx], clear = False)
# #         model.setObjective(diffabs[idx], clear = False)

    #绝对值
    for idx in data_result.index.unique():
        myratio = data_result.loc[idx,'ratio']
        class_2, org_id, size = idx
#         if myratio!=0:
#             print('==========',myratio)
        model.addCons(diff[idx] == X[class_2, size] - myratio)

        model.addCons(diffabs[idx] >= diff[idx])
        model.addCons(diffabs[idx] >= -1 * diff[idx])
        model.addCons(diffabs[idx] <= diff[idx] + M * (1 - dllb[idx]))
        model.addCons(diffabs[idx] <= -1 * diff[idx] + M * dllb[idx])
    
#     df = data_result.reset_index('size')
#     df_1 = data_result.reset_index(['org_id','size'])
#     for idx in df_1.index.unique():
#         class_2 = idx
#         model.addCons(quicksum(bl[class_2, size_1] for size_1 in df_1.loc[idx, ['size']].drop_duplicates().values.ravel()) >= 1)
    
    # 约束： 每个尺码的件数之和 等于 最小装箱数
#     for idx_s in df.index.unique():
#         class_2, org_id = idx_s
#         model.addCons(quicksum(X[class_2,size_1] for size_1 in df.loc[idx_s, ['size']].values.ravel()) == minput)

    
    model.setMinimize()
    model.optimize()
    status = model.getStatus()
    
    # 输出到出X为excle到本地
    OutX = {}
    if model.getStatus() =='optimal':
        for i in X:
            OutX[i] = round(model.getVal(X[i]))
#         for idx11 in diff:
#             print('diff--',idx, '= ', model.getVal(diff[idx]),"======",data_result.loc[idx, 'qty'])
    else:
        print("status",'*'*30, model.getStatus())
        print("obj:", model.getObjVal())
        print("gap:", model.getGap())
#         break
    return OutX

In [20]:
# shoes_data

In [21]:
shoes_data2 = shoes_data.copy()

In [22]:
shoes_data2.loc[shoes_data2['ratio'] ==0 ,'ratio'] =0.01
shoes_data2.loc[shoes_data2['qty'] ==0 ,'qty'] =0.01

In [23]:
shoes_data2['ratio']= shoes_data2['ratio']*100

In [24]:
# shoes_data2

In [25]:
shoes_result_2 =pd.DataFrame(columns=('小类','尺码','尺码比例'))

for class_name in class_arry:
    shoes_data_test = shoes_data2.loc[shoes_data2['class_2']== class_name]
    shoes_data_test = shoes_data_test.reset_index().drop('index',axis=1)
    #     print(clo_data_test)
    #     minput = BoxSize_dict[class_name]['minput']
    #     print('boxsize =======================',minput)   
    class_result = {}
    class_result = cal_shoes_model_2(shoes_data_test)
    liningResult = pd.DataFrame(class_result, index =['尺码比例']).T.rename_axis(['小类','尺码']).reset_index()
    liningResult['尺码比例'] = liningResult['尺码比例']/100
    print(liningResult)
    shoes_result_2 = pd.concat([shoes_result_2,liningResult],ignore_index=True)

      小类    尺码  尺码比例
0   减震跑鞋  10.0  0.10
1   减震跑鞋  10.5  0.28
2   减震跑鞋  11.0  0.11
3   减震跑鞋  11.5  0.01
4   减震跑鞋  12.0  0.07
5   减震跑鞋  12.5  0.01
6   减震跑鞋  13.0  0.02
7   减震跑鞋  13.5  0.01
8   减震跑鞋  14.0  0.01
9   减震跑鞋   4.5  0.00
10  减震跑鞋   5.0  0.05
11  减震跑鞋   5.5  0.06
12  减震跑鞋   6.0  0.19
13  减震跑鞋   6.5  0.47
14  减震跑鞋   7.0  0.39
15  减震跑鞋   7.5  0.92
16  减震跑鞋   8.0  1.00
17  减震跑鞋   8.5  0.32
18  减震跑鞋   9.0  0.92
19  减震跑鞋   9.5  0.54
       小类    尺码  尺码比例
0   户外休闲鞋  10.0  0.11
1   户外休闲鞋  10.5  0.39
2   户外休闲鞋  11.0  0.16
3   户外休闲鞋   4.5  0.01
4   户外休闲鞋   5.0  0.01
5   户外休闲鞋   5.5  0.02
6   户外休闲鞋   6.0  0.03
7   户外休闲鞋   6.5  0.15
8   户外休闲鞋   7.0  0.28
9   户外休闲鞋   7.5  0.67
10  户外休闲鞋   8.0  0.87
11  户外休闲鞋   8.5  0.24
12  户外休闲鞋   9.0  1.00
13  户外休闲鞋   9.5  0.65
       小类    尺码  尺码比例
0   潮流休闲鞋    10  0.14
1   潮流休闲鞋  10.5  0.36
2   潮流休闲鞋    11  0.14
3   潮流休闲鞋  11.5  0.01
4   潮流休闲鞋    12  0.03
5   潮流休闲鞋  12.5  0.01
6   潮流休闲鞋    13  0.01
7   潮流休闲鞋  13.5  0.01
8   潮流休闲鞋    14  0.01
9   潮流休闲鞋

In [26]:
shresult = pd.merge(result, shoes_result_2, how='right', on=['小类', '尺码'])

In [27]:
shresult

,小类,尺码,6尺码件数,12尺码件数,尺码比例
0,减震跑鞋,10,0,0,0.10
1,减震跑鞋,10.5,0,1,0.28
2,减震跑鞋,11,0,0,0.11
3,减震跑鞋,11.5,1,0,0.01
4,减震跑鞋,12,0,0,0.07
5,减震跑鞋,12.5,0,0,0.01
6,减震跑鞋,13,0,0,0.02
7,减震跑鞋,13.5,1,1,0.01
8,减震跑鞋,14,0,0,0.01
9,减震跑鞋,4.5,0,0,0.00


In [28]:
exl_time = dt.datetime.now().strftime('%Y%m%d')
locaurl ='C:\\Users\\liuju\\Desktop\\LiNingResult\\LiningShoesResult'+exl_time+'.xlsx'
shresult.to_excel(locaurl, index = False)